# Training and Evaluation Plan

This notebook is used for developing the training & evaluation steps that will be converted to Kubeflow components.  Since a pre-trained model is being used right now, there is no training, but ordinarily this would be where the training step would go, and if we do decide to fine-tune the model, it could be done here.

## Load the Pre-Trained Model from Hugging Face Hub

By initializing the model from a name string (all-MiniLM-L6-v2), sentence transformers knows to get the model from [the hub](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [75]:
import torch
import json
import typing as t
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from google.cloud import storage
from pathlib import Path
from subprocess import call
from scipy.spatial.distance import cdist
from tqdm import tqdm
from sklearn.metrics import ndcg_score, label_ranking_average_precision_score, roc_auc_score

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_NAME = "all-MiniLM-L6-v2"
MODEL = SentenceTransformer(MODEL_NAME, device=DEVICE)

if DEVICE == "cuda":
    torch.cuda.empty_cache()
print("Using device", DEVICE)

LOCAL_MODEL_PATH = "local_model"

PROJECT_ID = "queryable-docs-dev"
REGION = "us-central1"
GCS_BUCKET = "queryable-docs-artifacts-5024"
GCS_MODEL_FOLDER = f"{MODEL_NAME.lower()}"
GCS_MODEL_FOLDER_URI = f"gs://{GCS_BUCKET}/{GCS_MODEL_FOLDER}"
GCS_PIPELINE_ARTIFACTS_FOLDER = f"{GCS_BUCKET}/{MODEL_NAME.lower()}/training_and_eval_artifacts"
GCS_PIPELINE_ARTIFACTS_URI = f"gs://{GCS_PIPELINE_ARTIFACTS_FOLDER}"

LOCAL_DATA_FOLDER = "data"
GCS_DATA_FOLDER = "ir_eval_data"

GCS_CLIENT = storage.Client()

Using device cuda


## Fine-Tune the Model

We are not doing this, so here is a placeholder.

### Save the Model to GCS

If we were fine-tuning, we would want to save the weights.  Let's go through the motions anyway.

It would be nice to have a custom model hub, or to be able to save and load models directly to/from cloud storage.  But Hugging Face does not [and will not](https://github.com/huggingface/transformers/issues/23412) support that, so we will locally and then upload to cloud storage.

Saving to cloud storage requires iterating over the folder structure with the cloud storage Python API.  gsutil makes this much easier, but it must be called from the shell. Calling shell commands in Jupyter with subprocess is not straightforward, so we will just run commands directly here and make a function for using subprocess when we shift over to kubeflow. 

In [54]:
# save model locally first
MODEL.save(LOCAL_MODEL_PATH)

In [60]:
!gsutil cp -r ./local_model/* $GCS_MODEL_FOLDER_URI

Copying file://./local_model/1_Pooling/config.json [Content-Type=application/json]...
Copying file://./local_model/config.json [Content-Type=application/json]...     
Copying file://./local_model/config_sentence_transformers.json [Content-Type=application/json]...
Copying file://./local_model/modules.json [Content-Type=application/json]...    
- [4 files][  1.4 KiB/  1.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./local_model/pytorch_model.bin [Content-Type=application/octet-stream]...
Copying file://./local_model/README.md [Content-Type=text/markdown]...          
Copying file://./local_model/sentence_bert_config.json [Content-Type=application/json]...
Copying file://./local_model/special_tokens_map.json [Conten

In [61]:
!rm -R local_model

If the bucket must be deleted:

## Evaluate the Model

The next part of the pipeline will be the evaluation component, which will need to load the models and the data, and evaluate, and log metrics.

### Load the Model from GCS

If this notebook has been running from the start, the model will already be in memory.  But we should act as if these were micro-services, and the training service ends after dumping the model to storage.  The evaluation service should then read from storage instead of from memory.  We will do this because when we port this code over to Kubeflow, we will have to do it there.

First we will copy the files from cloud storage to a local folder.

In [5]:
def download_files_from_cloud_storage(
    gcs_client, 
    bucket: str, 
    bucket_folder: str, 
    local_folder: str,
    exclude_list: t.List[str] = None
) -> None:
    """Copies files from cloud storage folder to local folder."""
    bucket = gcs_client.get_bucket(bucket)
    blobs = bucket.list_blobs(prefix=bucket_folder)
    for blob in blobs:
        if blob.name.endswith("/"):
            continue
        path_split = blob.name.split("/")
        filename = path_split[-1]
        directory = "/".join(path_split[:-1])
        if exclude_list is not None and (directory in exclude_list or filename in exclude_list):
            print(f"Skipping artifact {blob.name}")
            next
        path = Path(f"{local_folder}/{directory}")
        path.mkdir(parents=True, exist_ok=True)
        print(f"Downloading {blob.name} to: {path}/{filename}\n")
        blob.download_to_filename(f"{path}/{filename}")

In [6]:
download_files_from_cloud_storage(
    gcs_client=GCS_CLIENT, 
    bucket=GCS_BUCKET, 
    bucket_folder=GCS_MODEL_FOLDER, 
    local_folder=LOCAL_MODEL_PATH,
    exclude_list=["training_and_eval_artifacts"]
)

In [3]:
# load the model from local path
MODEL = SentenceTransformer(f"{LOCAL_MODEL_PATH}/{GCS_MODEL_FOLDER}", device=DEVICE)

### Load the Eval Data from GCS

In [73]:
download_files_from_cloud_storage(
    gcs_client=GCS_CLIENT, 
    bucket=GCS_BUCKET, 
    bucket_folder=GCS_DATA_FOLDER, 
    local_folder=LOCAL_DATA_FOLDER
)

In [4]:
# load the eval data from local path
with open(f"{LOCAL_DATA_FOLDER}/{GCS_DATA_FOLDER}/corpus.json", "r") as f:
    corpus = json.load(f)
with open(f"{LOCAL_DATA_FOLDER}/{GCS_DATA_FOLDER}/queries.json", "r") as f:
    queries = json.load(f)
with open(f"{LOCAL_DATA_FOLDER}/{GCS_DATA_FOLDER}/qrels.json", "r") as f:
    qrels = json.load(f)


### Run Inference on Eval Data

In [80]:
MODEL.__dict__

{'_model_card_vars': {},
 '_model_card_text': '---\npipeline_tag: sentence-similarity\ntags:\n- sentence-transformers\n- feature-extraction\n- sentence-similarity\nlanguage: en\nlicense: apache-2.0\ndatasets:\n- s2orc\n- flax-sentence-embeddings/stackexchange_xml\n- MS Marco\n- gooaq\n- yahoo_answers_topics\n- code_search_net\n- search_qa\n- eli5\n- snli\n- multi_nli\n- wikihow\n- natural_questions\n- trivia_qa\n- embedding-data/sentence-compression\n- embedding-data/flickr30k-captions\n- embedding-data/altlex\n- embedding-data/simple-wiki\n- embedding-data/QQP\n- embedding-data/SPECTER\n- embedding-data/PAQ_pairs\n- embedding-data/WikiAnswers\n\n---\n\n\n# all-MiniLM-L6-v2\nThis is a [sentence-transformers](https://www.SBERT.net) model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.\n\n## Usage (Sentence-Transformers)\nUsing this model becomes easy when you have [sentence-transformers](https://www.SB

In [5]:
def create_embeddings(model, model_embed_dim: int, queries: dict, corpus: dict) -> t.Tuple[np.array]:
    """Creates embeddings for the query strings and texts"""
    q_embeddings = np.array(
        model.encode(list(queries.values())),
        dtype=np.float32
    ).reshape(-1, model_embed_dim)

    docs = [doc['text'] for doc in corpus.values()]
    d_embeddings = np.array(
        model.encode(docs),
        dtype=np.float32
    ).reshape(-1, model_embed_dim)

    return q_embeddings, d_embeddings

In [6]:
q_embed, d_embed = create_embeddings(model=MODEL, model_embed_dim=384, queries=queries, corpus=corpus)

In [7]:
print(q_embed.shape, d_embed.shape)

(400, 384) (40724, 384)


In [8]:
def rescale(d: float, dampening_factor: float = 1):
    """
    Inverts distance to be similarity.  Dividing by d + 1 ensures that 0 becomes 1 in the new scale.

    :param d: distance measurement
    :param dampening_factor: Increase this to flatten the curve and make larger values fall off slower 
        in the new scale.  This parameter should be tuned to what makes sense.  1.5 is a reasonable 
        starting value, see: https://www.desmos.com/calculator/eluirxagoz
    """
    return (1 / (d + 1)) ** (1 / dampening_factor)


def determine_clusters(dist_matrix: np.array, dist_threshold: float, text_ids: list = None):
    """
    Builds clusters of similar documents.

    :param text_ids: text IDs of documents to cluster, NOT the doc IDs
    :param dist_matrix: Pre-computed distance matrix between all docs
    :param dist_threshold: Documents with a Euclidean distance > this value are not clustered together.

    :return: tuple of number of clusters and dict mapping of doc_id to cluster
    """
    ag = AgglomerativeClustering(
        n_clusters=None,
        metric='precomputed',
        linkage='average',
        distance_threshold=dist_threshold
    )
    clusters = ag.fit_predict(dist_matrix)
    if text_ids is not None:
        clusters = {id: clusters[i] for i, id in enumerate(text_ids)}
    else:
        clusters = {id: clusters[i] for i, id in enumerate(range(len(clusters)))}
    return ag.n_clusters_, clusters


def compute_pairwise_dist_matrix(doc_embeddings: np.array):
    """Computes pairwise distance matrix for document clustering."""
    return cdist(doc_embeddings, doc_embeddings, metric='euclidean')


def score_relevance(query_vector: np.array, doc_embeddings: np.array, text_ids: list = None):
    """Computes scaled similarity (relevance) to a query vector for every document."""
    dist_to_query = cdist(query_vector.reshape(1, -1), doc_embeddings, metric='euclidean')
    dist_to_query = rescale(d=dist_to_query[0])
    if text_ids is not None:
        output = {text_ids[i]: dist_to_query[i] for i in range(len(text_ids))}
    else:
        output = {i: dist_to_query[i] for i in range(len(dist_to_query))}
    return output


def combine_labels_and_preds(qrels: dict, qrels_pred: dict, queries: dict, docs: dict) -> pd.DataFrame:
    # assemble the labeled data
    dfs = []
    for q, q_res in qrels.items():
        doc_ids = list(q_res)
        labels = list(q_res.values())
        q_name = [q] * len(doc_ids)
        q_text = queries[q]
        doc_texts = [docs[did]['text'] for did in doc_ids]
        qdf = pd.DataFrame({
            "query": q_name,
            "query_text": q_text,
            "doc_id": doc_ids,
            "doc_text": doc_texts,
            "label": labels,
        })
        qdf = qdf.sort_values(["label", "doc_id"], ascending=False).reset_index(drop=True)
        qdf['binary_label'] = np.where(qdf['label'] > 0, 1, 0)
        dfs.append(qdf)
    df = pd.concat(dfs, axis=0)

    # assemble the predictions
    pred_dfs = []
    for q, q_res in qrels_pred.items():
        doc_ids = list(q_res)
        scores = list(q_res.values())
        q_name = [q] * len(doc_ids)
        pdf = pd.DataFrame({"query": q_name, "doc_id": doc_ids, "similarity_score": scores})
        pred_dfs.append(pdf)
    pred_df = pd.concat(pred_dfs, axis=0)

    # combine the predictions with the labels
    final_df = pd.merge(left=df, right=pred_df, how='left', on=['query', 'doc_id'])
    final_df['similarity_score'].fillna(0, inplace=True)
    final_df.sort_values(["label", "similarity_score"], ascending=False, inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    return final_df


In [21]:
# map document IDs from the corpus to their index values in the corpus
doc_id_to_tid_map = {k: v for v, k in enumerate(list(corpus))}
tid_to_doc_id_map = {k: v for k, v in enumerate(list(corpus))}

# map query IDs from the queries to their index values
query_index_to_query_id_map = {k: v for v, k in enumerate(list(queries))}

In [15]:
# map query IDs to their embeddings
query_id_to_embed_map = {qid: q_embed[i,:] for i, qid in enumerate(list(queries))}

# map document IDs to their embeddings
doc_id_to_embedding_map = {did: d_embed[i,:] for i, did in enumerate(list(corpus))}

In [33]:
qrels_pred = {}
for query_id, doc_ids in tqdm(qrels.items()):
    # get the query embedding
    qe = query_id_to_embed_map[query_id].reshape(1, -1)
    
    # get the document embeddings for the docs found in this query's relations
    text_ids = [doc_id_to_tid_map[did] for did in list(doc_ids)]
    de = np.concatenate([doc_id_to_embedding_map[did] for did in list(doc_ids)], axis=0).reshape(-1, 384)
 
    # score the documents and map scores back to original doc IDs
    query_doc_scores = score_relevance(query_vector=qe, doc_embeddings=de, text_ids=text_ids)
    query_doc_scores = {tid_to_doc_id_map[k]: v for k, v in query_doc_scores.items()}
    
    # store results for this query
    q_res = {query_id: query_doc_scores}
    qrels_pred.update(q_res)


100%|██████████| 400/400 [00:00<00:00, 3997.49it/s]


In [72]:
def combine_labels_and_preds(qrels: dict, qrels_pred: dict, queries: dict, docs: dict) -> pd.DataFrame:
    # assemble the labeled data
    dfs = []
    for q, q_res in qrels.items():
        doc_ids = list(q_res)
        labels = list(q_res.values())
        q_name = [q] * len(doc_ids)
        q_text = queries[q]
        doc_texts = [docs[did]['text'] for did in doc_ids]
        qdf = pd.DataFrame({
            "query": q_name,
            "query_text": q_text,
            "doc_id": doc_ids,
            "doc_text": doc_texts,
            "label": labels,
        })
        qdf = qdf.sort_values(["label", "doc_id"], ascending=False).reset_index(drop=True)
        qdf['binary_label'] = np.where(qdf['label'] > 0, 1, 0)
        dfs.append(qdf)
    df = pd.concat(dfs, axis=0)

    # assemble the predictions
    pred_dfs = []
    for q, q_res in qrels_pred.items():
        doc_ids = list(q_res)
        scores = list(q_res.values())
        q_name = [q] * len(doc_ids)
        pdf = pd.DataFrame({"query": q_name, "doc_id": doc_ids, "similarity_score": scores})
        pred_dfs.append(pdf)
    pred_df = pd.concat(pred_dfs, axis=0)

    # combine the predictions with the labels
    final_df = pd.merge(left=df, right=pred_df, how='left', on=['query', 'doc_id'])
    final_df['similarity_score'].fillna(0, inplace=True)
    final_df.sort_values(["query", "label", "similarity_score"], ascending=False, inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    return final_df


def score_results(df: pd.DataFrame):
    """Split data into groups, score each query group's ranking by similarity score, and re-combine."""
    groups = [y for x, y in df.groupby('query')]
    for g in range(len(groups)):
        groups[g]['ndcg_top10'] = ndcg_score([groups[g]['label']], [groups[g]['similarity_score']], k=10)
        groups[g]['ndcg_top100'] = ndcg_score([groups[g]['label']], [groups[g]['similarity_score']], k=100)
        groups[g]['ndcg'] = ndcg_score([groups[g]['label']], [groups[g]['similarity_score']])
        groups[g]['lrap'] = label_ranking_average_precision_score(
            [groups[g]['binary_label']], [groups[g]['similarity_score']]
        )
        groups[g]['roc_auc'] = roc_auc_score(groups[g]['binary_label'], groups[g]['similarity_score'])
    return pd.concat(groups, axis=0)


In [73]:
df_pred = combine_labels_and_preds(
    qrels=qrels, 
    qrels_pred=qrels_pred, 
    queries=queries, 
    docs=corpus
)
df_pred.head()

,query,query_text,doc_id,doc_text,label,binary_label,similarity_score
0,TREC_Entity-9,Members of The Beaux Arts Trio.,<dbpedia:Bernard_Greenhouse>,"Bernard Greenhouse (January 3, 1916 – May 13, ...",2,1,0.525035
1,TREC_Entity-9,Members of The Beaux Arts Trio.,<dbpedia:Antônio_Meneses>,"Antônio Meneses Neto (born in Recife, 1957) is...",2,1,0.441961
2,TREC_Entity-9,Members of The Beaux Arts Trio.,<dbpedia:Menahem_Pressler>,"Menahem Pressler (born 16 December 1923, Magde...",2,1,0.436024
3,TREC_Entity-9,Members of The Beaux Arts Trio.,<dbpedia:Ida_Kavafian>,"Ida Kavafian (born October 29, 1952, Istanbul)...",2,1,0.433822
4,TREC_Entity-9,Members of The Beaux Arts Trio.,<dbpedia:Beaux_Arts_Trio>,The Beaux Arts Trio was a noted piano trio. Th...,1,1,0.535366


In [69]:
# make sure every query:doc pair has been accounted for
assert(len(df_pred) == sum([len(v) for v in qrels.values()]))

In [76]:
df = score_results(df=df_pred)
df.head()

,query,query_text,doc_id,doc_text,label,binary_label,similarity_score,ndcg_top10,ndcg_top100,ndcg,lrap,roc_auc
43384,INEX_LD-2009022,Szechwan dish food cuisine,<dbpedia:Suanla_chaoshou>,Suanla chaoshou is a dish of Szechuan cuisine ...,2,1,0.472756,0.194122,0.492381,0.599473,0.281605,0.449405
43385,INEX_LD-2009022,Szechwan dish food cuisine,<dbpedia:Hot_pot>,"Hot pot (also known as steamboat in Singapore,...",2,1,0.470632,0.194122,0.492381,0.599473,0.281605,0.449405
43386,INEX_LD-2009022,Szechwan dish food cuisine,<dbpedia:Chinese_cuisine>,Chinese cuisine includes styles originating fr...,2,1,0.469359,0.194122,0.492381,0.599473,0.281605,0.449405
43387,INEX_LD-2009022,Szechwan dish food cuisine,<dbpedia:Shuizhu>,Shuizhuroupian (Chinese: 水煮肉片; pinyin: shǔizhǔ...,2,1,0.466894,0.194122,0.492381,0.599473,0.281605,0.449405
43388,INEX_LD-2009022,Szechwan dish food cuisine,<dbpedia:Guoba>,"Guoba (鍋耙, 鍋巴, 锅巴, lit. ""pan adherents""), some...",2,1,0.466193,0.194122,0.492381,0.599473,0.281605,0.449405


In [70]:
# run check again
assert(len(df) == sum([len(v) for v in qrels.values()]))

In [77]:
# parameters and metrics to be logged
print("Param: queries", len(queries))
print("Param: documents", len(corpus))
print("Param: index_type", "flat")
print("Metric: average_ndcg_top10", df["ndcg_top10"].mean())
print("Metric: average_ndcg_top100", df["ndcg_top100"].mean())
print("Metric: average_ndcg", df["ndcg"].mean())
print("Metric: average_lrap", df["lrap"].mean())
print("Metric: average_roc_auc", df["roc_auc"].mean())

Param: queries 400
Param: documents 40724
Param: index_type flat
Metric: average_ndcg_top10 0.5132412198580375
Metric: average_ndcg_top100 0.6964808780625574
Metric: average_ndcg 0.7559563294596426
Metric: average_lrap 0.5947266872653154
Metric: average_roc_auc 0.7467223131260317


In [63]:
# check some categories
# baselines here: https://github.com/iai-group/DBpedia-Entity/
print(df[df['query'].str.contains("SemSearch_ES")]['ndcg_top10'].mean())
print(df[df['query'].str.contains("INEX_LD")]['ndcg_top10'].mean())
print(df[df['query'].str.contains("QALD2")]['ndcg_top10'].mean())


0.6216015288176907
0.48863184132554155
0.5221475182606636


In [71]:
list(queries)

['INEX_LD-20120111',
 'INEX_LD-20120121',
 'INEX_LD-20120122',
 'INEX_LD-20120131',
 'INEX_LD-20120211',
 'INEX_LD-20120221',
 'INEX_LD-20120231',
 'INEX_LD-20120232',
 'INEX_LD-20120311',
 'INEX_LD-20120312',
 'INEX_LD-20120321',
 'INEX_LD-20120331',
 'INEX_LD-20120332',
 'INEX_LD-20120411',
 'INEX_LD-20120412',
 'INEX_LD-20120421',
 'INEX_LD-20120422',
 'INEX_LD-20120431',
 'INEX_LD-20120511',
 'INEX_LD-20120512',
 'INEX_LD-20120521',
 'INEX_LD-20120522',
 'INEX_LD-20120531',
 'INEX_LD-20120532',
 'INEX_LD-2009022',
 'INEX_LD-2009039',
 'INEX_LD-2009053',
 'INEX_LD-2009061',
 'INEX_LD-2009062',
 'INEX_LD-2009063',
 'INEX_LD-2009074',
 'INEX_LD-2009115',
 'INEX_LD-2010004',
 'INEX_LD-2010014',
 'INEX_LD-2010019',
 'INEX_LD-2010020',
 'INEX_LD-2010037',
 'INEX_LD-2010043',
 'INEX_LD-2010057',
 'INEX_LD-2010069',
 'INEX_LD-2010100',
 'INEX_LD-2010106',
 'INEX_LD-2012301',
 'INEX_LD-2012303',
 'INEX_LD-2012305',
 'INEX_LD-2012309',
 'INEX_LD-2012311',
 'INEX_LD-2012313',
 'INEX_LD-201231